In [8]:
import pandas as pd
import numpy as np 

df = pd.read_csv("data_2014.csv", sep =',', skiprows=5)
df.columns= ['time', 'record', 'Gb', 'G', 'Gd', 'uv', 'T', 'phi', 'v','p']
df.time = pd.to_datetime(df['time'], format= '%d/%m/%Y %H:%M')
df.set_index(df['time'], inplace=True)
df["Gb"] = df['Gb'].apply(pd.to_numeric, errors='coerce')
df.dtypes
df.head()

,time,record,Gb,G,Gd,uv,T,phi,v,p
time,,,,,,,,,,
2014-01-01 00:10:00,2014-01-01 00:10:00,15875.0,0.538,0.024,0.001,0.005,17.84,57.27,2.641,877.5115
2014-01-01 00:20:00,2014-01-01 00:20:00,15876.0,0.413,0.008,0.000,0.006,17.77,57.44,2.658,877.5121
2014-01-01 00:30:00,2014-01-01 00:30:00,15877.0,0.275,0.028,0.000,0.003,17.50,56.28,2.569,877.5287
2014-01-01 00:40:00,2014-01-01 00:40:00,15878.0,0.412,0.008,0.000,0.007,17.87,56.28,1.884,877.5046
2014-01-01 00:50:00,2014-01-01 00:50:00,15879.0,1.168,0.007,0.000,0.007,18.21,55.97,2.183,877.6179


In [30]:
dt = 10* 60
mes = 6
dfmes = df[ df.index.month == mes]
dias = dfmes.index.day.unique().values
H= []
for dia in dias:
    dfw = dfmes[dfmes.index.day ==dia]
    aux = dt * dfw.G.sum()/1E6
    H.append(aux)

dfw = pd.DataFrame({'dia':dias, 'H':H})
dfw.head()

,dia,H
0,1,27.339788
1,2,27.382765
2,3,22.462196
3,4,23.471299
4,5,22.388032


In [31]:
# G0 [W/m2]
dfw["G0"] = 1367 * (1 + 0.033 * np.cos(dias*2*np.pi/365))
dfw.head()

,dia,H,G0
0,1,27.339788,1412.104316
1,2,27.382765,1412.084267
2,3,22.462196,1412.050859
3,4,23.471299,1412.004101
4,5,22.388032,1411.944007


In [32]:
from ps import d2r, doy, B, delta, EdT, dtiempo, hms, omega, theta_z
# Calculo de cos(theta_z)
D = 1
M = mes
Y = 2014
tp = 0
v = 0
h = 0
m = 0
s = 0
lat = 18.8291
Lloc = 99.23
Lst = 6 *15

In [34]:
lat = d2r(lat)
costheta = [ ]

for dia in dias:
    N = doy(Y, M, dia)
    b = B(N)
    d = delta(N)
    E = EdT(b)
    Dt = dtiempo(E, Lst, Lloc, v)
    theta = [ ]
    for hora in range(0, 24, 1):
        for minuto in range(0, 60, 10):
            HMS = hms(hora, minuto, s)
            w = omega(HMS, Dt, tp)
            tz = theta_z(lat, d, w)
            theta.append(tz)
    x = np.cos(theta)
    x[x < 0] = 0
    aux = sum(x) *dt
    costheta.append(aux)

dfw["costhetaDt"] = costheta
dfw["H0"] = dfw["G0"] * costheta / 1E6
dfw["Kt"] = dfw["H"]/dfw["H0"]
dfw.head()
dfw.to_csv("")

,dia,H,G0,costhetaDt,H0,Kt
0,1,27.339788,1412.104316,25582.061654,36.124540,0.756820
1,2,27.382765,1412.084267,25558.296471,36.090468,0.758726
2,3,22.462196,1412.050859,25535.560414,36.057510,0.622955
3,4,23.471299,1412.004101,25513.880031,36.025703,0.651515
4,5,22.388032,1411.944007,25493.280613,35.995085,0.621975
